In [19]:
from elasticsearch import Elasticsearch
import os
import pandas as pd

es = Elasticsearch(
    ['http://localhost:9200'],  # Adresse Elasticsearch
    basic_auth=('elastic', 'IcuM24+R*=HgaV*3WWTE')
)


In [20]:
# Définition de la structure de l'index
index_name = 'bano_data'
index_body = {
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},  # id unique
            "numero": {"type": "text"},  # numéro dans la voie (ex: 1, 1BIS)
            "voie": {"type": "text"},  # nom de la voie
            "code_post": {"type": "keyword"},  # code postal (5 caractères)
            "nom_comm": {"type": "text"},  # nom de la commune
            "source": {"type": "keyword"},  # source (OSM, OD, etc.)
            "lat": {"type": "float"},  # latitude en degrés décimaux
            "lon": {"type": "float"},  # longitude en degrés décimaux
            "location": {"type": "geo_point"}  # champ pour la géolocalisation
        }
    }
}

In [21]:
# Création de l'index
es.indices.create(index=index_name, body=index_body)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'bano_data'})

## Vérification dans la console Kibana de la création du nouvel Index:
# GET _cat/indices?v

In [ ]:
folder_path = "bano_data"

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        csv_file_path = os.path.join(folder_path, file)
        # Lire le fichier CSV
        df = pd.read_csv(csv_file_path, sep=",", names=[
            "id", "number", "street_name", "zipcode", "city", "source", "latitude", "longitude"
        ])

        # Supprimer les colonnes inutiles
        df.drop(columns=["message", "@version", "@timestamp", "host"], inplace=True, errors='ignore')

        # Convertir les champs latitude et longitude en float
        df["latitude"] = pd.to_numeric(df["latitude"], errors='coerce')
        df["longitude"] = pd.to_numeric(df["longitude"], errors='coerce')



        # Envoyer les données transformées à Elasticsearch
        for index, row in df.iterrows():
            doc_id = row['id']
            document = {
                                "id": row['id'],
                                "numero": row['number'],
                                "voie": row['street_name'],
                                "code_post": row['zipcode'],
                                "nom_comm": row['city'],
                                "source": row['source'],
                                "lat": row['latitude'],
                                "lon": row['longitude'],
                                "location": {
                                    "lat": row['latitude'],
                                    "lon": row['longitude']
                                }
                            }
            es.index(index="bano_data", id=doc_id, document=document)

print("Données ingérées avec succès!")

In [ ]:
# Exécuter la commande _cat/indices
response = es.cat.indices(index="bano*", h="index,docs.count,store.size", v=True)

# Afficher le résultat
print(response)

# Recherche par rapport à une position GPS donnée
### Retourne uniquement le document avec la position la plus proche

In [ ]:
# Exécuter la requête
response = es.search(index="bano*", body={
  "size": 1, 
  "sort": [
    {
      "_geo_distance": {
        "location": {
          "lat": 46.746586,
          "lon": 5.907748
        }
      }
    }
  ]
})

# Afficher le résultat
print(response['hits']['hits'][0]['_source'])
